# RNN : LSTM


In [1]:
# import
import pickle 

import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
# from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV


# reproductibility
np.random.seed(42)

In [2]:
## ICI ON RÈGLE QUELLE TAILLE DE FENETRE ON VEUT UTILISER (POUR NE PAS DUPLIQUER LE CODE)
taille_fenetre_to_run = "courte"
assert taille_fenetre_to_run in ["courte", "moyenne", "longue"]

In [3]:
if taille_fenetre_to_run == "courte":
    run_split = pickle.load(open("Data/splits_courte.pkl", "rb"))
elif taille_fenetre_to_run == "moyenne":
    run_split = pickle.load(open("Data/splits_moyenne.pkl", "rb"))
else:
    run_split = pickle.load(open("Data/splits_longue.pkl", "rb"))

In [4]:
len(run_split)

5

In [5]:
# utiser binary encoding dans RNN 

split_0 = run_split[0]
split_1 = run_split[1]
split_2 = run_split[2]
split_3 = run_split[3]
split_4 = run_split[4]

# Vérifier qu'ils ont tous les mêmes clefs
print(split_0.keys())
print(split_1.keys())
print(split_2.keys())
print(split_3.keys())
print(split_4.keys())

dict_keys(['X_train_np_label', 'X_train_np_binary', 'y_train_np', 'X_train_df_label', 'X_train_df_binary', 'y_train_df', 'X_test_np_label', 'X_test_np_binary', 'y_test_np', 'X_test_df_label', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_label', 'X_train_np_binary', 'y_train_np', 'X_train_df_label', 'X_train_df_binary', 'y_train_df', 'X_test_np_label', 'X_test_np_binary', 'y_test_np', 'X_test_df_label', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_label', 'X_train_np_binary', 'y_train_np', 'X_train_df_label', 'X_train_df_binary', 'y_train_df', 'X_test_np_label', 'X_test_np_binary', 'y_test_np', 'X_test_df_label', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_label', 'X_train_np_binary', 'y_train_np', 'X_train_df_label', 'X_train_df_binary', 'y_train_df', 'X_test_np_label', 'X_test_np_binary', 'y_test_np', 'X_test_df_label', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_label', 'X_train_np_binary', 'y_train_np', 'X_train_df_label', 'X_train_d

In [6]:
# Drop window_tipe, X_train_label, X_test_label, X_train_df_label, X_test_df_label
for s in [split_0, split_1, split_2, split_3, split_4]:
    s.pop('X_train_np_label')
    s.pop('X_test_np_label')
    s.pop('X_train_df_label')
    s.pop('X_test_df_label')
    print(s.keys())


dict_keys(['X_train_np_binary', 'y_train_np', 'X_train_df_binary', 'y_train_df', 'X_test_np_binary', 'y_test_np', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_binary', 'y_train_np', 'X_train_df_binary', 'y_train_df', 'X_test_np_binary', 'y_test_np', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_binary', 'y_train_np', 'X_train_df_binary', 'y_train_df', 'X_test_np_binary', 'y_test_np', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_binary', 'y_train_np', 'X_train_df_binary', 'y_train_df', 'X_test_np_binary', 'y_test_np', 'X_test_df_binary', 'y_test_df'])
dict_keys(['X_train_np_binary', 'y_train_np', 'X_train_df_binary', 'y_train_df', 'X_test_np_binary', 'y_test_np', 'X_test_df_binary', 'y_test_df'])


In [7]:
# Vérifier que pour toutes les colonnes, chaque valeur min et max est 0 et 1

# Vérifier pour chaque split
for i, split in enumerate(run_split):
    print(f"Vérification pour split_{i}:")
    X_train_binary, X_test_binary = split['X_train_np_binary'], split['X_test_np_binary']

    min_train = np.min(X_train_binary, axis=(0,1))  # Min pour chaque feature
    max_train = np.max(X_train_binary, axis=(0,1))  # Max pour chaque feature
    min_test = np.min(X_test_binary, axis=(0,1))  # Min pour chaque feature
    max_test = np.max(X_test_binary, axis=(0,1))  # Max pour chaque feature

    # print(f"Min train: {min_train}")
    # print(f"Max train: {max_train}")
    # print(f"Min test: {min_test}")
    # print(f"Max test: {max_test}")

    assert np.all(min_train >= 0), "Min train < 0"
    assert np.all(min_train <= 1), "Min train > 1"
    assert np.all(max_train >= 0), "Max train < 0"
    assert np.all(max_train <= 1), "Max train > 1"
    assert np.all(min_test >= 0), "Min test < 0"
    assert np.all(min_test <= 1), "Min test > 1"
    assert np.all(max_test >= 0), "Max test < 0"
    assert np.all(max_test <= 1), "Max test > 1"



Vérification pour split_0:
Vérification pour split_1:
Vérification pour split_2:
Vérification pour split_3:
Vérification pour split_4:


In [8]:
def create_lstm(input_shape, units=100, dropout_rate=0.2, activation = 'tanh', learning_rate = 0.001):

    # pour ajouter des couches 
    model = Sequential()

    # units : 100, Plus ce nombre est élevé, plus le modèle peut capturer de relations complexes dans les données, mais cela augmente aussi le coût computationnel.
    # activation : tanh, fonction d'activation tanh (classique dans les LSTM)
    model.add(LSTM(units, input_shape=input_shape, activation=activation))

    # éviter surapprentissage
    model.add(Dropout(dropout_rate)) 

    # output pour un problème de régresssion 
    model.add(Dense(1)) 

    # optimizer adam 
    # mse : typique pour un problème de régression
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse') # mse pour un problème de régression ?

    return model

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [10]:
scores = []

param_grid = {
    'model__units': [50, 100, 200],
    'dropout_rate': [0.2, 0.3, 0.5],
    'model__activation': ['tanh', 'relu'],
    'learning_rate': [0.001, 0.01, 0.1], 
    'batch_size': [32, 64, 128], 
    'epochs': [100, 200, 300]
}

for i, split in enumerate(run_split):
    print(f"Split_{i}")

    X_train, y_train = split['X_train_np_binary'], split['y_train_np']
    X_test, y_test = split['X_test_np_binary'], split['y_test_np']

    input_shape = X_train.shape[1:]
    print(f'Input shape: {input_shape}')

    model = KerasRegressor(build_fn=create_lstm, input_shape=input_shape, verbose=1) # , units=100, dropout_rate=0.2, activation='tanh', learning_rate=0.001, epochs=100, batch_size=32, verbose=1)
    # model = create_lstm(input_shape)

    gridsearch = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1) # attention j'ai déjà de la validation croisée moi, donc pas de cv ici  

    gridsearch.fit(X_train, y_train, validation_data=(X_test, y_test), verbose=1) # callbacks=[early_stopping], verbose=1)

    # model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping], verbose=1)

    # model_filename = f"Data/lstm_model_{taille_fenetre_to_run}_split_{i}.h5"
    # model.save(model_filename)

    # score = model.evaluate(X_test, y_test, verbose=0)
    # scores.append(score)

    print(f"Best params: {gridsearch.best_params_}")
    print(f"Best score: {gridsearch.best_score_}")

    best_model = gridsearch.best_estimator_
    best_model_filename = f"Data/lstm_model_{taille_fenetre_to_run}_split_{i}.h5"
    best_model.model.save(best_model_filename)

    score = best_model.score(X_test, y_test)
    scores.append(score)

Split_0
Input shape: (50, 25)


/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The KerasRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
print(f"Performance de chaque split: {scores}")
print(f"Performance moyenne: {np.mean(scores)}")

In [12]:

# random forest : label encoding
# régresssion linéaire : binary encoding